In [5]:
import pandas as pd
from openai import OpenAI
import numpy as np

In [7]:
 dfS = pd.read_csv('/Users/kruthikhulisandra/Desktop/SpotifyProject/streaming2125.csv', low_memory=False)

In [3]:
print(dfS.dtypes)

ts                                    object
platform                              object
ms_played                              int64
conn_country                          object
ip_addr                               object
master_metadata_track_name            object
master_metadata_album_artist_name     object
master_metadata_album_album_name      object
spotify_track_uri                     object
episode_name                          object
episode_show_name                     object
spotify_episode_uri                   object
audiobook_title                       object
audiobook_uri                         object
audiobook_chapter_uri                 object
audiobook_chapter_title               object
reason_start                          object
reason_end                            object
shuffle                                 bool
skipped                                 bool
offline                                 bool
offline_timestamp                    float64
incognito_

In [4]:
# Rename columns to remove 'master_metadata'
dfS.rename(columns=lambda x: x.replace('master_metadata_', ''), inplace=True)
dfS.rename(columns={
    'album_artist_name': 'artist_name',
    'album_album_name': 'album_name'
}, inplace=True)

# Display updated column names
print(dfS.columns)


Index(['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr', 'track_name',
       'artist_name', 'album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'audiobook_title',
       'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title',
       'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
       'offline_timestamp', 'incognito_mode'],
      dtype='object')


In [5]:
dfS['spotify_track_uri'] = dfS['spotify_track_uri'].apply(
    lambda uri: uri.split(':')[-1] if isinstance(uri, str) else None
)
# Rename the column
dfS.rename(columns={'spotify_track_uri': 'spotifyID'}, inplace=True)

dfS


,ts,platform,ms_played,conn_country,ip_addr,track_name,artist_name,album_name,spotifyID,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2021-11-01T16:02:42Z,"iOS 15.0.1 (iPhone14,5)",6826,US,128.6.37.130,Drive,CHASE WRIGHT,Drive,4aRAElCM6ve3OISZnU7adb,NaN,...,NaN,NaN,NaN,playbtn,logout,True,False,False,NaN,False
1,2021-11-01T16:45:45Z,OS X 11.6.0 [x86 8],146517,US,128.6.37.236,I Found You,Cash Cash,I Found You,3wYOGJYD31sLRmBgCvWxa4,NaN,...,NaN,NaN,NaN,playbtn,endplay,False,False,False,NaN,True
2,2021-11-01T16:53:18Z,OS X 11.6.0 [x86 8],409733,US,128.6.37.236,Gravity,yetep,Gravity,2rOfnCrbxZpMvENtx8LqY9,NaN,...,NaN,NaN,NaN,clickrow,trackdone,False,False,False,NaN,True
3,2021-11-01T16:56:59Z,OS X 11.6.0 [x86 8],219671,US,128.6.37.236,Easy On Me,Adele,Easy On Me,0gplL1WMoJ6iYaPgMCL0gX,NaN,...,NaN,NaN,NaN,trackdone,fwdbtn,True,False,False,NaN,True
4,2021-11-01T18:15:30Z,OS X 11.6.0 [x86 8],247735,US,128.6.37.236,Zombie,Bad Wolves,Zombie,1vNoA9F5ASnlBISFekDmg3,NaN,...,NaN,NaN,NaN,fwdbtn,remote,True,False,False,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55463,2025-01-14T20:21:14Z,ios,229210,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Taste (feat. Offset),Tyga,Legendary,2toVe5hfuIi97ytDPDbQFt,NaN,...,NaN,NaN,NaN,fwdbtn,trackdone,True,False,False,1.736886e+09,False
55464,2025-01-14T20:24:40Z,ios,212800,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Crazy (feat. Casey Cook),Wooli,Crazy (feat. Casey Cook),61ilfNBj7SUVn3YYQlWy04,NaN,...,NaN,NaN,NaN,trackdone,trackdone,True,False,False,1.736886e+09,False
55465,2025-01-14T20:26:32Z,ios,119320,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,hate me like u love me,Syence,hate me like u love me,42WEv3u02DH3mWGbYEZKLW,NaN,...,NaN,NaN,NaN,trackdone,fwdbtn,True,True,False,1.736886e+09,False
55466,2025-01-14T20:29:38Z,ios,187012,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Knowing How To Break (with RUNN),Last Heroes,Knowing How To Break (with RUNN),3cLJYCnJVVAWZgPcAafv6u,NaN,...,NaN,NaN,NaN,trackerror,trackdone,True,False,False,1.736886e+09,False


In [6]:
dfS['ts'] = pd.to_datetime(dfS['ts'])

# Create separate columns for date and time
dfS['date'] = dfS['ts'].dt.date  # Extract the date part
dfS['time'] = dfS['ts'].dt.time  # Extract the time part

columns = ['date', 'time'] + [col for col in dfS.columns if col not in ['date', 'time']]
dfS = dfS[columns]

dfS

,date,time,ts,platform,ms_played,conn_country,ip_addr,track_name,artist_name,album_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2021-11-01,16:02:42,2021-11-01 16:02:42+00:00,"iOS 15.0.1 (iPhone14,5)",6826,US,128.6.37.130,Drive,CHASE WRIGHT,Drive,...,NaN,NaN,NaN,playbtn,logout,True,False,False,NaN,False
1,2021-11-01,16:45:45,2021-11-01 16:45:45+00:00,OS X 11.6.0 [x86 8],146517,US,128.6.37.236,I Found You,Cash Cash,I Found You,...,NaN,NaN,NaN,playbtn,endplay,False,False,False,NaN,True
2,2021-11-01,16:53:18,2021-11-01 16:53:18+00:00,OS X 11.6.0 [x86 8],409733,US,128.6.37.236,Gravity,yetep,Gravity,...,NaN,NaN,NaN,clickrow,trackdone,False,False,False,NaN,True
3,2021-11-01,16:56:59,2021-11-01 16:56:59+00:00,OS X 11.6.0 [x86 8],219671,US,128.6.37.236,Easy On Me,Adele,Easy On Me,...,NaN,NaN,NaN,trackdone,fwdbtn,True,False,False,NaN,True
4,2021-11-01,18:15:30,2021-11-01 18:15:30+00:00,OS X 11.6.0 [x86 8],247735,US,128.6.37.236,Zombie,Bad Wolves,Zombie,...,NaN,NaN,NaN,fwdbtn,remote,True,False,False,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55463,2025-01-14,20:21:14,2025-01-14 20:21:14+00:00,ios,229210,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Taste (feat. Offset),Tyga,Legendary,...,NaN,NaN,NaN,fwdbtn,trackdone,True,False,False,1.736886e+09,False
55464,2025-01-14,20:24:40,2025-01-14 20:24:40+00:00,ios,212800,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Crazy (feat. Casey Cook),Wooli,Crazy (feat. Casey Cook),...,NaN,NaN,NaN,trackdone,trackdone,True,False,False,1.736886e+09,False
55465,2025-01-14,20:26:32,2025-01-14 20:26:32+00:00,ios,119320,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,hate me like u love me,Syence,hate me like u love me,...,NaN,NaN,NaN,trackdone,fwdbtn,True,True,False,1.736886e+09,False
55466,2025-01-14,20:29:38,2025-01-14 20:29:38+00:00,ios,187012,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Knowing How To Break (with RUNN),Last Heroes,Knowing How To Break (with RUNN),...,NaN,NaN,NaN,trackerror,trackdone,True,False,False,1.736886e+09,False


In [7]:
def simplify_platform(platform):
    platform = str(platform).lower()  # Normalize to lowercase
    if platform in ['ios', 'osx', 'android', 'not_applicable']:
        return platform  # Preserve already correct values
    elif 'ios' in platform:
        return 'ios'
    elif 'os x' in platform:
        return 'osx'
    elif 'android' in platform:
        return 'android'
    elif 'not_applicable' in platform:
        return 'not_applicable'
    else:
        return 'unknown'

dfS['platform'] = dfS['platform'].apply(simplify_platform)

dfS

,date,time,ts,platform,ms_played,conn_country,ip_addr,track_name,artist_name,album_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2021-11-01,16:02:42,2021-11-01 16:02:42+00:00,ios,6826,US,128.6.37.130,Drive,CHASE WRIGHT,Drive,...,NaN,NaN,NaN,playbtn,logout,True,False,False,NaN,False
1,2021-11-01,16:45:45,2021-11-01 16:45:45+00:00,osx,146517,US,128.6.37.236,I Found You,Cash Cash,I Found You,...,NaN,NaN,NaN,playbtn,endplay,False,False,False,NaN,True
2,2021-11-01,16:53:18,2021-11-01 16:53:18+00:00,osx,409733,US,128.6.37.236,Gravity,yetep,Gravity,...,NaN,NaN,NaN,clickrow,trackdone,False,False,False,NaN,True
3,2021-11-01,16:56:59,2021-11-01 16:56:59+00:00,osx,219671,US,128.6.37.236,Easy On Me,Adele,Easy On Me,...,NaN,NaN,NaN,trackdone,fwdbtn,True,False,False,NaN,True
4,2021-11-01,18:15:30,2021-11-01 18:15:30+00:00,osx,247735,US,128.6.37.236,Zombie,Bad Wolves,Zombie,...,NaN,NaN,NaN,fwdbtn,remote,True,False,False,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55463,2025-01-14,20:21:14,2025-01-14 20:21:14+00:00,ios,229210,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Taste (feat. Offset),Tyga,Legendary,...,NaN,NaN,NaN,fwdbtn,trackdone,True,False,False,1.736886e+09,False
55464,2025-01-14,20:24:40,2025-01-14 20:24:40+00:00,ios,212800,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Crazy (feat. Casey Cook),Wooli,Crazy (feat. Casey Cook),...,NaN,NaN,NaN,trackdone,trackdone,True,False,False,1.736886e+09,False
55465,2025-01-14,20:26:32,2025-01-14 20:26:32+00:00,ios,119320,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,hate me like u love me,Syence,hate me like u love me,...,NaN,NaN,NaN,trackdone,fwdbtn,True,True,False,1.736886e+09,False
55466,2025-01-14,20:29:38,2025-01-14 20:29:38+00:00,ios,187012,US,2600:1001:b001:94fd:fdfb:b746:1d53:d114,Knowing How To Break (with RUNN),Last Heroes,Knowing How To Break (with RUNN),...,NaN,NaN,NaN,trackerror,trackdone,True,False,False,1.736886e+09,False


In [8]:
#delete rows of AudioBooks and Podcasts
nonSongs = [
    'episode_name', 'episode_show_name', 'spotify_episode_uri',
    'audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri',
    'audiobook_chapter_title'
]
rows_to_drop = dfS[dfS[nonSongs].notnull().any(axis=1)].index

# Drop those rows
dfS.drop(rows_to_drop, inplace=True)




In [9]:
dfS.drop(columns= nonSongs, inplace=True) #drop all the non-song columns
dfS.drop(columns=['offline_timestamp'], inplace=True)
dfS.drop(columns=['ip_addr'], inplace=True)
dfS.drop(columns=['ts'], inplace=True)
dfS

,date,time,platform,ms_played,conn_country,track_name,artist_name,album_name,spotifyID,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
0,2021-11-01,16:02:42,ios,6826,US,Drive,CHASE WRIGHT,Drive,4aRAElCM6ve3OISZnU7adb,playbtn,logout,True,False,False,False
1,2021-11-01,16:45:45,osx,146517,US,I Found You,Cash Cash,I Found You,3wYOGJYD31sLRmBgCvWxa4,playbtn,endplay,False,False,False,True
2,2021-11-01,16:53:18,osx,409733,US,Gravity,yetep,Gravity,2rOfnCrbxZpMvENtx8LqY9,clickrow,trackdone,False,False,False,True
3,2021-11-01,16:56:59,osx,219671,US,Easy On Me,Adele,Easy On Me,0gplL1WMoJ6iYaPgMCL0gX,trackdone,fwdbtn,True,False,False,True
4,2021-11-01,18:15:30,osx,247735,US,Zombie,Bad Wolves,Zombie,1vNoA9F5ASnlBISFekDmg3,fwdbtn,remote,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55463,2025-01-14,20:21:14,ios,229210,US,Taste (feat. Offset),Tyga,Legendary,2toVe5hfuIi97ytDPDbQFt,fwdbtn,trackdone,True,False,False,False
55464,2025-01-14,20:24:40,ios,212800,US,Crazy (feat. Casey Cook),Wooli,Crazy (feat. Casey Cook),61ilfNBj7SUVn3YYQlWy04,trackdone,trackdone,True,False,False,False
55465,2025-01-14,20:26:32,ios,119320,US,hate me like u love me,Syence,hate me like u love me,42WEv3u02DH3mWGbYEZKLW,trackdone,fwdbtn,True,True,False,False
55466,2025-01-14,20:29:38,ios,187012,US,Knowing How To Break (with RUNN),Last Heroes,Knowing How To Break (with RUNN),3cLJYCnJVVAWZgPcAafv6u,trackerror,trackdone,True,False,False,False


In [26]:
desired_order = [
    'date', 'time', 'conn_country', 'platform', 'track_name', 
    'artist_name', 'album_name', 'ms_played', 'spotifyID', 
    'reason_start', 'reason_end', 'shuffle', 'skipped', 
    'offline', 'incognito_mode'
]

# Reorder the columns in the DataFrame
dfS = dfS[desired_order]
dfS

,date,time,conn_country,platform,track_name,artist_name,album_name,ms_played,spotifyID,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
0,2021-11-01,16:02:42,US,ios,Drive,CHASE WRIGHT,Drive,6826,4aRAElCM6ve3OISZnU7adb,playbtn,logout,True,False,False,False
1,2021-11-01,16:45:45,US,osx,I Found You,Cash Cash,I Found You,146517,3wYOGJYD31sLRmBgCvWxa4,playbtn,endplay,False,False,False,True
2,2021-11-01,16:53:18,US,osx,Gravity,yetep,Gravity,409733,2rOfnCrbxZpMvENtx8LqY9,clickrow,trackdone,False,False,False,True
3,2021-11-01,16:56:59,US,osx,Easy On Me,Adele,Easy On Me,219671,0gplL1WMoJ6iYaPgMCL0gX,trackdone,fwdbtn,True,False,False,True
4,2021-11-01,18:15:30,US,osx,Zombie,Bad Wolves,Zombie,247735,1vNoA9F5ASnlBISFekDmg3,fwdbtn,remote,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55463,2025-01-14,20:21:14,US,ios,Taste (feat. Offset),Tyga,Legendary,229210,2toVe5hfuIi97ytDPDbQFt,fwdbtn,trackdone,True,False,False,False
55464,2025-01-14,20:24:40,US,ios,Crazy (feat. Casey Cook),Wooli,Crazy (feat. Casey Cook),212800,61ilfNBj7SUVn3YYQlWy04,trackdone,trackdone,True,False,False,False
55465,2025-01-14,20:26:32,US,ios,hate me like u love me,Syence,hate me like u love me,119320,42WEv3u02DH3mWGbYEZKLW,trackdone,fwdbtn,True,True,False,False
55466,2025-01-14,20:29:38,US,ios,Knowing How To Break (with RUNN),Last Heroes,Knowing How To Break (with RUNN),187012,3cLJYCnJVVAWZgPcAafv6u,trackerror,trackdone,True,False,False,False


In [34]:
# Save dfS to a CSV file
dfS.to_csv('dfSCleaned.csv', index=False)


In [11]:
 dfS = pd.read_csv('/Users/kruthikhulisandra/Desktop/SpotifyProject/dfSCleaned.csv', low_memory=False)

In [21]:
list(dfS.columns)

['date',
 'time',
 'conn_country',
 'platform',
 'track_name',
 'artist_name',
 'album_name',
 'ms_played',
 'spotifyID',
 'reason_start',
 'reason_end',
 'shuffle',
 'skipped',
 'offline',
 'incognito_mode']

In [19]:
dfS = dfS.drop(columns=['year'])


In [25]:
dfS['date'] = pd.to_datetime(dfS['date'])  # Ensure 'date' column is in datetime format
dfS24 = dfS[dfS['date'].dt.year == 2024]



In [27]:
dfS24

,date,time,conn_country,platform,track_name,artist_name,album_name,ms_played,spotifyID,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
39381,2024-01-01,02:23:44,US,ios,Tears,Helix,Tears,236000,6yqxQhLv5iJ2hLGmN4iHlG,playbtn,trackdone,True,False,False,False
39382,2024-01-01,02:24:56,US,ios,Tonight,NURKO,Tonight,48560,3BP1klbHxsOf6IxscNIX0r,trackdone,unexpected-exit-while-paused,True,False,False,False
39383,2024-01-01,18:54:57,US,ios,"For Aisha (Featured in ""The Sky Is Pink"")",MEMBA,"For Aisha (Featured in ""The Sky Is Pink"")",16370,6rfahvufEQDIVTHJIU2QQB,clickrow,endplay,False,True,False,False
39384,2024-01-01,18:55:00,US,ios,Sink Your Teeth In,Flux Pavilion,.wav,3134,1Y7UkYVReK4z5hgm0121SS,playbtn,fwdbtn,True,True,False,False
39385,2024-01-01,18:58:14,US,ios,2U,David Guetta,2U,194906,762UXiwEHYY97VL45sAUPR,fwdbtn,trackdone,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54983,2024-12-31,16:54:34,US,osx,Redo,Crankdat,Redo,197260,6NZunfz5wbY8pYeDDlJUas,trackdone,trackdone,True,False,False,False
54984,2024-12-31,16:58:23,US,osx,Walk On Water (Wooli & Trivecta Remix),SLANDER,Thrive (The Remixes),228413,0XlU2Ar7l9y52nveQb22Xd,trackdone,trackdone,True,False,False,False
54985,2024-12-31,17:02:53,US,osx,You Found Me,Adventure Club,You Found Me,269538,5caU54QlqCMPEKIbQ3nbhd,trackdone,trackdone,True,False,False,False
54986,2024-12-31,17:05:14,US,osx,F______ LOVE YOU,ALLEYCVT,F______ LOVE YOU,140999,2LNZWTaw7gf1hZTjOXvZQC,trackdone,trackdone,True,False,False,False


In [31]:
# Group by the song's unique identifier and count the occurrences
song_play_counts = dfS24.groupby(['spotifyID', 'track_name', 'artist_name']).size().reset_index(name='play_count')

# Filter for songs played more than 2 times
played_songs = song_play_counts[song_play_counts['play_count'] > 2]
played_songs = played_songs[~played_songs.apply(lambda row: row.astype(str).str.contains(r'\bASMR\b', case=False).any(), axis=1)]


df_songs = played_songs.sort_values(by='play_count', ascending=False)
df_songs

,spotifyID,track_name,artist_name,play_count
3567,5TQDtOeQ3IU2VqKbCn6Rwq,Falling Apart,ARMNHMR,50
3900,60lVHKWmvmsJlv14Xw0mdO,Divine,ZEXTONE,50
4872,7k5uBcPwecGOf90m2vpxJF,Gone,Hoang,42
790,19m23w2ANVhtB7rApM6pbN,Back To U,SLANDER,39
4306,6evviQsK6E43H91dZIHDi3,IF THIS IS A DREAM,k?d,38
...,...,...,...,...
1938,2vm52EstTRxGsvQ44gEFRL,Superhero,Audien,3
923,1NO185IYAdEw9qp4b1DLNF,BREATHE,devin,3
2516,3qOuySPLpVyBXLuTNMgbRj,On Hold - Jamie xx Remix,The xx,3
3660,5eV0O3aTblLHPQYh5MnPHc,everything... and more,devin,3


In [43]:
# Group by the song's unique identifier and count the occurrences
song_play_counts = dfS24.groupby(['spotifyID', 'track_name', 'artist_name']).size().reset_index(name='play_count')

# Filter for songs played more than 2 times

played_songs = played_songs[~played_songs.apply(lambda row: row.astype(str).str.contains(r'\bASMR\b', case=False).any(), axis=1)]
played_songs = song_play_counts[(song_play_counts['play_count'] >= 5)]


df_songs = played_songs.sort_values(by='play_count', ascending=False)
df_songs

,spotifyID,track_name,artist_name,play_count
3567,5TQDtOeQ3IU2VqKbCn6Rwq,Falling Apart,ARMNHMR,50
3900,60lVHKWmvmsJlv14Xw0mdO,Divine,ZEXTONE,50
4872,7k5uBcPwecGOf90m2vpxJF,Gone,Hoang,42
790,19m23w2ANVhtB7rApM6pbN,Back To U,SLANDER,39
4306,6evviQsK6E43H91dZIHDi3,IF THIS IS A DREAM,k?d,38
...,...,...,...,...
2595,3ypw0T4LARAjt0sMJaCuoP,Loose Off The WHOGO Interlude (feat. Kamal Wil...,Rayana Jay,5
1244,1r7WUr8dw048EcIXA90lg2,Fireball,Marshmello,5
2620,41CZOSesgTuEkaoRKoBbzX,Running After Us,yetep,5
1197,1nSfvLVhsozMHS7rCF1BTi,Rose Colored Lenses,Miley Cyrus,5


In [53]:



client = OpenAI(api_key="<Open API Key>")

def analyze_song(track_name, artist_name):
    prompt = f"""
    Given the song "{track_name}" by {artist_name}, provide:
    1. A genre that captures the essence of the song.
    2. An energy level as a number between 0 and 1 based on the beats per minute.
    3. One of these themes: Uplifting, Nostalgic, Romantic, Bittersweet, Melancholy, Sentimental, Euphoric, Peaceful, Hopeful, Empowered.
    4. The language of this song
    Respond in the format:
    Genre: <genre>
    Energy: <energy>
    Theme: <theme>
    Language: <language>
    """
    try:
        response = client.chat.completions.create(
            model="o1-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        # Check if 'choices' is present and has elements
        if response.choices and len(response.choices) > 0:
            content = response.choices[0].message.content
            return content
        else:
            print(f"No choices returned for track: {track_name} by {artist_name}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [55]:
# Store results
results = []

# Iterate over all rows of df_songs
for _, row in df_songs.iterrows():
    track_name = row['track_name']
    artist_name = row['artist_name']
    
    # Analyze the song
    response = analyze_song(track_name, artist_name)
    
    # Initialize variables
    genre = energy = theme = language = None
    
    # Parse the response if it exists
    if response:
        lines = response.split("\n")
        for line in lines:
            if line.startswith("Genre:"):
                parts = line.split(":", 1)
                if len(parts) > 1:
                    genre = parts[1].strip()
            elif line.startswith("Energy:"):
                parts = line.split(":", 1)
                if len(parts) > 1:
                    try:
                        energy = float(parts[1].strip())
                    except ValueError:
                        print(f"Invalid energy value in response for track: {track_name} by {artist_name}")
            elif line.startswith("Theme:"):
                parts = line.split(":", 1)
                if len(parts) > 1:
                    theme = parts[1].strip()
            elif line.startswith("Language:"):
                parts = line.split(":", 1)
                if len(parts) > 1:
                    language = parts[1].strip()
    
    # Check if all data is extracted
    if genre is None or energy is None or theme is None or language is None:
        print(f"Missing information in response for track: {track_name} by {artist_name}")
        continue  # Skip appending incomplete responses
    
    # Append results
    results.append({
        "track_name": track_name,
        "artist_name": artist_name,
        "genre": genre,
        "energy": energy,
        "theme": theme,
        "language": language
    })



Missing information in response for track: Losing You by yetep
Missing information in response for track: First Place by yetep
Missing information in response for track: Worlds Away by yetep
Missing information in response for track: Us in Motion by Łaszewo
Missing information in response for track: Universe by JAMEO
Missing information in response for track: Max Out! by LittGloss
Invalid energy value in response for track: Burning by Seth Hills
Missing information in response for track: Burning by Seth Hills
Missing information in response for track: People Change by Hoang
Missing information in response for track: Pages by Naliya
Missing information in response for track: In the Night by Pindell
Invalid energy value in response for track: Shanivaar Raati (Remix By Dj Chetas) by Arijit Singh
Missing information in response for track: Shanivaar Raati (Remix By Dj Chetas) by Arijit Singh
Missing information in response for track: Woman Like Me by Adele
Missing information in response fo

In [57]:
# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Save or display results
df_results.to_csv("song_analysis.csv", index=False)  # Save the results to a CSV file

# Merge results with df_songs
df_songs = df_songs.merge(df_results, on=['track_name', 'artist_name'], how='left')

# Display the merged dataframe or save it
print(df_songs)  # Display the merged dataframe
df_songs.to_csv("dfAn24.csv", index=False)  # Save the merged dataframe to a new CSV


                  spotifyID  \
0    5TQDtOeQ3IU2VqKbCn6Rwq   
1    60lVHKWmvmsJlv14Xw0mdO   
2    7k5uBcPwecGOf90m2vpxJF   
3    19m23w2ANVhtB7rApM6pbN   
4    6evviQsK6E43H91dZIHDi3   
..                      ...   
980  3ypw0T4LARAjt0sMJaCuoP   
981  1r7WUr8dw048EcIXA90lg2   
982  41CZOSesgTuEkaoRKoBbzX   
983  1nSfvLVhsozMHS7rCF1BTi   
984  48pyUxg9RYuwie5M9R7qRn   

                                            track_name   artist_name  \
0                                        Falling Apart       ARMNHMR   
1                                               Divine       ZEXTONE   
2                                                 Gone         Hoang   
3                                            Back To U       SLANDER   
4                                   IF THIS IS A DREAM           k?d   
..                                                 ...           ...   
980  Loose Off The WHOGO Interlude (feat. Kamal Wil...    Rayana Jay   
981                                           Fireb

In [65]:
df_merged = dfS24.merge(df_songs, on='spotifyID', how='inner')


,date,time,conn_country,platform,track_name_x,artist_name_x,album_name,ms_played,spotifyID,reason_start,...,skipped,offline,incognito_mode,track_name_y,artist_name_y,play_count,genre,energy,theme,language
0,2024-01-01,18:58:14,US,ios,2U,David Guetta,2U,194906,762UXiwEHYY97VL45sAUPR,fwdbtn,...,False,False,False,2U,David Guetta,6,Dance-pop,0.50,Romantic,English
1,2024-01-01,18:58:21,US,ios,Toyota Music,Big Sean,Hall Of Fame,2484,5Pd7oO5U5fP9lXUjeUFMxi,fwdbtn,...,True,False,False,Toyota Music,Big Sean,12,Hip-Hop/Rap,0.50,Empowered,English
2,2024-01-01,19:01:40,US,ios,Meant for Two,Miles Away,Miles Away,200000,1Dho6Tu3ko1AEHzJWTvI0K,fwdbtn,...,False,False,False,Meant for Two,Miles Away,8,J-Pop/Electronic,0.70,Romantic,Japanese
3,2024-01-01,19:06:28,US,ios,Too Hotty,Quality Control,Quality Control: Control The Streets Volume 1,295997,3hWUIMNBwLNug1O8o2PPRW,trackdone,...,False,False,False,Too Hotty,Quality Control,9,Hip-Hop/Trap,0.80,Empowered,English
4,2024-01-01,19:09:39,US,ios,Drifting Away,Audien,Drifting Away,198750,6nbXYLeP21ksEQNFrHl2a2,trackdone,...,False,False,False,Drifting Away,Audien,15,Progressive House,0.47,Uplifting,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9805,2024-12-29,21:21:55,US,ios,Falling Apart,ARMNHMR,Falling Apart,62864,5TQDtOeQ3IU2VqKbCn6Rwq,fwdbtn,...,True,False,False,Falling Apart,ARMNHMR,50,Future Bass,0.70,Melancholy,English
9806,2024-12-30,17:28:27,US,ios,THREE IN THE MORNING,k?d,CAGE SCRIPT,170476,36iU4gfHhUbjwk88kZAwgW,trackdone,...,False,False,False,THREE IN THE MORNING,k?d,6,Future Bass,0.75,Melancholy,English
9807,2024-12-30,17:31:26,US,ios,Lost My Way,WHIPPED CREAM,Lost My Way,186857,5Euq8gNFRLyd75tGhpXKNC,trackdone,...,False,False,False,Lost My Way,WHIPPED CREAM,6,Alternative Rock,0.60,Melancholy,English
9808,2024-12-31,16:47:42,US,osx,dashstar* - VIP,Knock2,dashstar* (VIP),140481,4FKRT4uPFx2L4exy9DCs1o,trackdone,...,False,False,False,dashstar* - VIP,Knock2,6,Hip-Hop,0.85,Empowered,English


In [69]:
# Drop the unnecessary columns 'track_name_y' and 'artist_name_y'
df_merged = df_merged.drop(columns=['track_name_y', 'artist_name_y'])

# Rename the columns 'track_name_x' and 'artist_name_x' to 'track_name' and 'artist_name'
df_merged = df_merged.rename(columns={'track_name_x': 'track_name', 'artist_name_x': 'artist_name'})

# Remove rows with any null values
df_merged = df_merged.dropna()

In [71]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9594 entries, 0 to 9809
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            9594 non-null   datetime64[ns]
 1   time            9594 non-null   object        
 2   conn_country    9594 non-null   object        
 3   platform        9594 non-null   object        
 4   track_name      9594 non-null   object        
 5   artist_name     9594 non-null   object        
 6   album_name      9594 non-null   object        
 7   ms_played       9594 non-null   int64         
 8   spotifyID       9594 non-null   object        
 9   reason_start    9594 non-null   object        
 10  reason_end      9594 non-null   object        
 11  shuffle         9594 non-null   bool          
 12  skipped         9594 non-null   bool          
 13  offline         9594 non-null   bool          
 14  incognito_mode  9594 non-null   bool          
 15  play_coun

In [73]:
df_merged.to_csv('df24.csv')

In [7]:
df = pd.read_csv('/Users/kruthikhulisandra/Desktop/SpotifyProject/df24.csv') #2024 dataframe

In [9]:
df

,Unnamed: 0,date,time,conn_country,platform,track_name,artist_name,album_name,ms_played,spotifyID,...,reason_end,shuffle,skipped,offline,incognito_mode,play_count,genre,energy,theme,language
0,0,2024-01-01,18:58:14,US,ios,2U,David Guetta,2U,194906,762UXiwEHYY97VL45sAUPR,...,trackdone,True,False,False,False,6,Dance-pop,0.50,Romantic,English
1,1,2024-01-01,18:58:21,US,ios,Toyota Music,Big Sean,Hall Of Fame,2484,5Pd7oO5U5fP9lXUjeUFMxi,...,fwdbtn,True,True,False,False,12,Hip-Hop/Rap,0.50,Empowered,English
2,2,2024-01-01,19:01:40,US,ios,Meant for Two,Miles Away,Miles Away,200000,1Dho6Tu3ko1AEHzJWTvI0K,...,trackdone,True,False,False,False,8,J-Pop/Electronic,0.70,Romantic,Japanese
3,3,2024-01-01,19:06:28,US,ios,Too Hotty,Quality Control,Quality Control: Control The Streets Volume 1,295997,3hWUIMNBwLNug1O8o2PPRW,...,trackdone,True,False,False,False,9,Hip-Hop/Trap,0.80,Empowered,English
4,4,2024-01-01,19:09:39,US,ios,Drifting Away,Audien,Drifting Away,198750,6nbXYLeP21ksEQNFrHl2a2,...,trackdone,True,False,False,False,15,Progressive House,0.47,Uplifting,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589,9805,2024-12-29,21:21:55,US,ios,Falling Apart,ARMNHMR,Falling Apart,62864,5TQDtOeQ3IU2VqKbCn6Rwq,...,endplay,True,True,False,False,50,Future Bass,0.70,Melancholy,English
9590,9806,2024-12-30,17:28:27,US,ios,THREE IN THE MORNING,k?d,CAGE SCRIPT,170476,36iU4gfHhUbjwk88kZAwgW,...,trackdone,True,False,False,False,6,Future Bass,0.75,Melancholy,English
9591,9807,2024-12-30,17:31:26,US,ios,Lost My Way,WHIPPED CREAM,Lost My Way,186857,5Euq8gNFRLyd75tGhpXKNC,...,trackdone,True,False,False,False,6,Alternative Rock,0.60,Melancholy,English
9592,9808,2024-12-31,16:47:42,US,osx,dashstar* - VIP,Knock2,dashstar* (VIP),140481,4FKRT4uPFx2L4exy9DCs1o,...,trackdone,True,False,False,False,6,Hip-Hop,0.85,Empowered,English


In [13]:
df.drop(columns=['Unnamed: 0'], inplace=True)


df['avg_played_energyPD'] = df['date'].map(df[df['skipped'] == 0].groupby('date')['energy'].mean())


df['avg_skipped_energyPD'] = df['date'].map(df[df['skipped'] == 1].groupby('date')['energy'].mean())


df['energy_diff_PD'] = df['avg_played_energyPD'] - df['avg_skipped_energyPD']

# Ensure 'time' is in proper datetime format
df['time'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))

# Extract hour of the day
df['hour'] = df['time'].dt.hour

#Add part_of_day
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'
        
#create column for time of day using the function just created
df['time_of_day'] = df['hour'].apply(get_part_of_day)

#This transformation captures the cyclical nature of hourly data, 
#Midnight = -1 , 6am = 0, Noon = +1, 6pm = 0
df['sin_hour'] = np.sin(2 * np.pi * (df['hour'] - 6) / 24)

df


df['sin_hour'] = df['sin_hour'].round(4)

df[['hour', 'sin_hour']].drop_duplicates().sort_values(by='hour')

#df.to_csv('dfAn2.csv', index=False)

,hour,sin_hour
348,0,-1.0000
349,1,-0.9659
12,2,-0.8660
14,3,-0.7071
33,4,-0.5000
71,5,-0.2588
155,6,0.0000
216,7,0.2588
222,8,0.5000
240,9,0.7071


In [15]:
df

,date,time,conn_country,platform,track_name,artist_name,album_name,ms_played,spotifyID,reason_start,...,genre,energy,theme,language,avg_played_energyPD,avg_skipped_energyPD,energy_diff_PD,hour,time_of_day,sin_hour
0,2024-01-01,2024-01-01 18:58:14,US,ios,2U,David Guetta,2U,194906,762UXiwEHYY97VL45sAUPR,fwdbtn,...,Dance-pop,0.50,Romantic,English,0.602000,0.550000,0.052000,18,evening,0.0000
1,2024-01-01,2024-01-01 18:58:21,US,ios,Toyota Music,Big Sean,Hall Of Fame,2484,5Pd7oO5U5fP9lXUjeUFMxi,fwdbtn,...,Hip-Hop/Rap,0.50,Empowered,English,0.602000,0.550000,0.052000,18,evening,0.0000
2,2024-01-01,2024-01-01 19:01:40,US,ios,Meant for Two,Miles Away,Miles Away,200000,1Dho6Tu3ko1AEHzJWTvI0K,fwdbtn,...,J-Pop/Electronic,0.70,Romantic,Japanese,0.602000,0.550000,0.052000,19,evening,-0.2588
3,2024-01-01,2024-01-01 19:06:28,US,ios,Too Hotty,Quality Control,Quality Control: Control The Streets Volume 1,295997,3hWUIMNBwLNug1O8o2PPRW,trackdone,...,Hip-Hop/Trap,0.80,Empowered,English,0.602000,0.550000,0.052000,19,evening,-0.2588
4,2024-01-01,2024-01-01 19:09:39,US,ios,Drifting Away,Audien,Drifting Away,198750,6nbXYLeP21ksEQNFrHl2a2,trackdone,...,Progressive House,0.47,Uplifting,English,0.602000,0.550000,0.052000,19,evening,-0.2588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589,2024-12-29,2024-12-29 21:21:55,US,ios,Falling Apart,ARMNHMR,Falling Apart,62864,5TQDtOeQ3IU2VqKbCn6Rwq,fwdbtn,...,Future Bass,0.70,Melancholy,English,0.620889,0.607353,0.013536,21,night,-0.7071
9590,2024-12-30,2024-12-30 17:28:27,US,ios,THREE IN THE MORNING,k?d,CAGE SCRIPT,170476,36iU4gfHhUbjwk88kZAwgW,trackdone,...,Future Bass,0.75,Melancholy,English,0.675000,NaN,NaN,17,evening,0.2588
9591,2024-12-30,2024-12-30 17:31:26,US,ios,Lost My Way,WHIPPED CREAM,Lost My Way,186857,5Euq8gNFRLyd75tGhpXKNC,trackdone,...,Alternative Rock,0.60,Melancholy,English,0.675000,NaN,NaN,17,evening,0.2588
9592,2024-12-31,2024-12-31 16:47:42,US,osx,dashstar* - VIP,Knock2,dashstar* (VIP),140481,4FKRT4uPFx2L4exy9DCs1o,trackdone,...,Hip-Hop,0.85,Empowered,English,0.825000,NaN,NaN,16,afternoon,0.5000
